In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
column_names = ['age', 'sex','cp','trestbps','chol','fbs','resteccg','thalach','exange','oldpeak','slope','ca','thall', 'num']
df = pd.read_csv(r"C:\Users\zeyne\Desktop\updated.data",header=None ,names=column_names ,na_values='?')

x=df.drop(['num' ,'ca','thall','slope'],axis=1).values         #Dataframe değil de numpy array formatında olması için sonuna .values ekledim
y=df['num']

df

In [ ]:
print(df.isnull().sum())

In [ ]:
x

Taking care of missing data

In [ ]:
y

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan , strategy='mean')
imputer.fit(x[: ,3:10])                                              #bu adımda sadece ortalama değer hesaplanır
x[: ,3:10]=imputer.transform(x[: ,3:10])                             #Eksik değerleri doldurulan dataset, dönüştürülmüş dataset olarak döndürülür.  SimpleImputer kullanırken np.array işlemi gerekli değildir.  doğrudan dönüştürülmüş veriyi döndürdüğü için, bu dönüştürülmüş veriyi doğrudan kullanabilirsiniz.
x

In [19]:
from plotly.express import scatter
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

# PCA modeli oluşturma
pca = PCA(n_components=2)
x_pca = pca.fit_transform(x_scaled)
pca_df = pd.DataFrame(x_pca, columns=["x", "y"])

y_df=pd.DataFrame(y, columns=["num"])

plot_df = pd.concat(objs=[y_df, pca_df], axis=1)

scatter(data_frame=plot_df, x='x', y='y', color='num', height=500)

In [ ]:
class_counts = df['num'].value_counts()
class_counts

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)                               #split den sonda yapsan daha iyi olabilir, y ye de uygulaman gerekeilir

# Veri setini bölme
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=100)
# Random Forest Classifier modelini oluşturma ve eğitme
class_weights = {0:1, 1:1, 2:2, 3:2, 4:8}
rf_classifier =RandomForestClassifier(class_weight=class_weights, random_state=100)
rf_classifier.fit(x_train, y_train)

# Test verisi üzerinde modelin performansını değerlendirme
y_pred = rf_classifier.predict(x_test)

# Sınıflandırma raporunu oluşturma
report = classification_report(y_test, y_pred)
print("RandomForestClassifier Sınıflandırma Raporu:")
report

WUHUUUUUU ACC YÜKSELDİ, SMOTE SAYESİNDE:

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
x_resampled, y_resampled = smote.fit_resample(x_train, y_train)

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

# Polinom özelliklerini oluşturma
poly = PolynomialFeatures(degree=6)
X_train_poly = poly.fit_transform(x_resampled)

# Veri setini bölme
x_train, x_test, y_train, y_test = train_test_split(X_train_poly, y_resampled, test_size=0.2, random_state=100)
# Random Forest Classifier modelini oluşturma ve eğitme
class_weights = {0:1, 1:1, 2:2, 3:2, 4:8}
rf_classifier =RandomForestClassifier(class_weight=class_weights, random_state=100)
rf_classifier.fit(x_train, y_train)

# Test verisi üzerinde modelin performansını değerlendirme
y_pred = rf_classifier.predict(x_test)

# Sınıflandırma raporunu oluşturma
report = classification_report(y_test, y_pred)
print("RandomForestClassifier Sınıflandırma Raporu:")
print(report)

RandomForestClassifier Sınıflandırma Raporu:
              precision    recall  f1-score   support

           0       0.72      0.67      0.69        63
           1       0.70      0.64      0.67        69
           2       0.82      0.79      0.81        68
           3       0.73      0.90      0.80        59
           4       0.96      0.94      0.95        69

    accuracy                           0.79       328
   macro avg       0.78      0.79      0.78       328
weighted avg       0.79      0.79      0.78       328



In [ ]:
from sklearn.model_selection import GridSearchCV

# GridSearchCV için parametrelerin olası değerlerini belirleyin
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# GridSearchCV nesnesini oluşturun
grid_search = GridSearchCV(RandomForestClassifier(class_weight=class_weights, random_state=100), param_grid, cv=5, scoring='accuracy')

# Eğitim veri seti üzerinde GridSearchCV'yi uygulayın
grid_search.fit(x_train, y_train)

# En iyi parametreleri ve en iyi skoru görüntüleme
print("En iyi parametreler:", grid_search.best_params_)
print("En iyi skor:", grid_search.best_score_)

# En iyi model
best_rf_classifier = grid_search.best_estimator_

# En iyi modeli kullanarak tahmin yapma
y_pred_best = best_rf_classifier.predict(x_test);

# Sınıflandırma raporunu oluşturma
report_best = classification_report(y_test, y_pred_best);
print("En iyi Random Forest Classifier Sınıflandırma Raporu:");
print(report_best);

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

# Veri setini bölmek
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=100)
# Karar ağacı modelini oluşturma
decision_tree_model = DecisionTreeClassifier(class_weight='balanced')

# Karar ağacı modelini eğitme
decision_tree_model.fit(x_train, y_train)

# Test verisi üzerinde modelin performansını değerlendirme
y_pred = decision_tree_model.predict(x_test)

# Sınıflandırma raporunu oluşturma
report = classification_report(y_test, y_pred)
print("DecisionTreeClassifier Sınıflandırma Raporu:")
print(report)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Gradient Boosting Classifier modelini oluşturma ve eğitme
gb_classifier = GradientBoostingClassifier()
gb_classifier.fit(x_train, y_train)

# Test verisi üzerinde modelin performansını değerlendirme
y_pred = gb_classifier.predict(x_test)

# Sınıflandırma raporunu oluşturma
report = classification_report(y_test, y_pred)
print("GradientBoostingClassifier Sınıflandırma Raporu:")
print(report)

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import classification_report

# Destek Vektör Makineleri modelini oluşturun ve eğitin

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=100)

svm_model = SVC()  # Lineer çekirdek kullanıyoruz
svm_model.fit(x_train, y_train)   # eğitiyotuz verilerimizle

# Test seti üzerinde tahmin
y_pred = svm_model.predict(x_test)

# Sınıflandırma raporunu oluşturma
report = classification_report(y_test, y_pred)
print("SVC Sınıflandırma Raporu:")
print(report)

svc deki bu uyarıların sebebi , datasetimizde class dağılımları dengesiz , bazı class lar için çok az sayıda instance var, ve svc buna dayanıklı bir yapıya sahip değil, bazı class lar için değerlendirmede yetersiz kalıp bu uyarıları veriyor


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

# Hiperparametre aralıklarını belirleme
param_grid = {
    'C': [0.1, 1, 10, 100],  # C parametresi için değerler
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Kernel tipi için seçenekler
    'degree': [2, 3, 4],  # Polinom çekirdeği için dereceler
    'gamma': ['scale', 'auto', 0.1, 1]  # Gamma parametresi için değerler
}

# SVC modeli oluşturma
svm_model = SVC(kernel='cuda')

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=100)

# GridSearchCV veya RandomizedSearchCV modeli oluşturma ve eğitme
grid_search = GridSearchCV(svm_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_train, y_train)

# En iyi parametreleri ve en iyi doğruluk skorunu bulma
print("En iyi parametreler:", grid_search.best_params_)
print("En iyi doğruluk skoru:", grid_search.best_score_)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# KNN modelini oluşturma ve eğitme
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(x_train, y_train)

# Modelin performansını test veri setiyle değerlendirme
y_pred = knn_model.predict(x_test)

# Sınıflandırma raporunu oluşturma
report = classification_report(y_test, y_pred)
print("KNeighborsClassifier Sınıflandırma Raporu:")
print(report)

Sınıflandırma Raporu:
              precision    recall  f1-score   support

           0       0.72      0.83      0.77        63
           1       0.65      0.63      0.64        49
           2       0.55      0.50      0.52        22
           3       0.33      0.30      0.32        23
           4       0.33      0.14      0.20         7

    accuracy                           0.62       164
   macro avg       0.52      0.48      0.49       164
weighted avg       0.61      0.62      0.61       164

